# Import relevant functions

In [1]:
import numpy as np
import random
import math
#from scipy.stats import spearmanr, pearsonr

In [2]:
#import the functions in the file MFunctions as mf
import MFunctions as mf

## Observables object

In [3]:
#Simple set of observables, only linear
set1=mf.observables("Linear.txt")
#Access the files which contain the given observables with filelist
print(set1.filelist)
#Access the stored observables with obslist
set1.obslist

Linear.txt


['np.trace(W) # \\sum_{i} W_{ii}', 'np.sum(W) # \\ sum_{i,j} W_{ij}']

In [4]:
#A more involved set with both linear and quadratic observables
set2=mf.observables(["Linear.txt","Quadratic.txt"])
print(set2.filelist)
print(set2.obslist)

['Linear.txt', 'Quadratic.txt']
['np.trace(W) # \\sum_{i} W_{ii}', 'np.sum(W) # \\ sum_{i,j} W_{ij}', 'np.sum(W*W) # sum{i,j} M_ij M_ij', 'np.sum(W*W.T) # sum{i,j} M_ij M_ji', 'np.sum(np.diagonal(W)*W.T) # \\sum_{i,j} W_{ii} W_{ij}', 'np.sum(np.diagonal(W)*W) # \\sum_{i,j} W_{ii} W_{ji}', 'np.sum(np.dot(W.T,W)) # sum{i,j,k} W_ij W_ik', 'np.sum(np.dot(W,W.T)) # sum{i,j,k} W_ij W_kj', 'np.sum(np.dot(W,W)) # sum{i,j,k} W_ij W_jk', 'np.sum(W) ** 2 #sum{i,j,k,l} W_ij W_kl', 'np.trace(W*W) # sum{i} W^2_ii', 'np.trace(W) ** 2  # \\sum_{i,j,k} W_{ii} W_{jj}', 'np.sum(W) * np.trace(W)  # \\sum_{i,j,k} W_{ii} W_{jk}']


In [5]:
#If something which is not a file or list of files is given as init an error message is returned
test = mf.observables(5)

Unknown initialisation value for the function observables, plese check input and retry.


## Vector class

In [6]:
#The observables objects are used in a further class called vector to build the vector of numeric observables
#starting from a given matrix
testmatrix = np.array([[1,2,3],[4,5,6],[7,8,9]])
#the class vector needs a word to which the vector will be associated, a matrix and an observables object
testvector = mf.vector("testword",testmatrix,set1)
#Numeric values are accessed through the method val
print(testvector.val)
#To access the files from which the observables have been imported use filelist
print(testvector.filelist)
#access the word to which the vector corresponds with word
testvector.word

[15 45]
Linear.txt


'testword'

## Vector dictionary

In [7]:
#In general we deal not with a single vector but a whole set of matrices corresponding to different verbs and stored
#in a file. We define a vector dictionary object which requiress an input file and an observables object,
#it then loads the matrices one by one building the corresponding vector for each
objdict = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set2)

In [8]:
# An additional optional argument can be given which specifies the position of file names whose matrices need to be transposed
subjdict = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set2,[0])

In [9]:
#It is also possible to produce vector dictionaries of mixed matrices. To do so instead of a single file name input a list of two elements,
# the first being a list of file names and the second an equally long list of weights (works for arbitrary length, files need not be only 2)
obj08subj02_dict = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set2,[1])
obj02subj08_dict = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set2,[1])

In [10]:
#vector_dictionary is a dictionary so standard dictionary operations apply. Take keys for example
#extract a random sample of keys
somesample = random.sample(list(objdict),5)
print(somesample)
#Dispaly numeric vector associated to given key
objdict[somesample[0]].val

['drain', 'embarrass', 'disintegrate', 'collect', 'grind']


array([ 7.80509197e-02, -1.47129790e-02,  5.87214866e+00,  6.32856306e-02,
        2.60356026e-02, -1.37565633e-02,  4.74017317e+00,  3.89849567e+00,
       -9.49467755e-01,  2.16471753e-04,  5.53481355e-02,  6.09194607e-03,
       -1.14836155e-03])

In [11]:
#Now we have the collection of vectors associated to all the matrices in a given file and computed using
#a set of observables. We use it to compare to SymVerb3500
#verb_symilarity is a class which requires a file name as init and simply creates a dictionary with keys "SYNONIME", "NONE"
#and so on, with associates values a list of pairs of vectors with their similarity
# N.B. this function requires an input file which has he same format as SimVerb3500
testsym = mf.verb_similarity("SimVerb-3500.txt")
#Print the keys
print(testsym.keys())
#To these keys are associated lists of the following form
testsym['SYNONYMS'][0]

dict_keys(['SYNONYMS', 'COHYPONYMS', 'ANTONYMS', 'HYPER/HYPONYMS', 'NONE'])


['take', 'remove', 6.81]

## Inner products

In [13]:
# There are 5 different types of inner products, each of which requires the observable-value vectors as input
# and possibly further arguments depending on which product is considered. For example to compute the inner
# product of observable deviation vetors we need to also specify the standard deviation and mean of the dictionary
mf.M_vec_mult_dev(objdict['drain'].val,objdict['collect'].val,objdict.mean,objdict.std)

-2.9787026754467782

## Averaged product

In [12]:
# averaged_product takes as input a verb_similarity dictionary object, a key in that dictionary,
# and a vector_dictionary object. It returns the mean and standard deviation of the vector products associated
# to the pairs in testsym and computed from the vectors in obj08subj02_dict. The fourth argument is optional and
# Specifies the type of inner product
# N.B. Here some care is needed since specifying the product also specifies whether we use observable-value
# or observable-deviation vectors. For example std_dev is cosine distance with observable-deviation vectors
print(mf.averaged_product(testsym,"SYNONYMS",obj08subj02_dict))
#Again but with observable deviation vectors
mf.averaged_product(testsym,"SYNONYMS",obj08subj02_dict,"std_dev")

[0.48117138614238786, 0.2673178579740687]


[0.3061805670475563, 0.4951254885623443]

In [14]:
# It is also possible to get a list of the inner products of the pairs instead of averaging over them
mf.averaged_product_list(testsym,"SYNONYMS",obj08subj02_dict,"std_dev")

{('take', 'remove'): 0.48169530166383223,
 ('shine', 'polish'): 0.7680022156292539,
 ('bake', 'broil'): 0.86947632333204,
 ('decay', 'decompose'): 0.89328708560628,
 ('jump', 'skip'): 0.45360071901881394,
 ('go', 'move'): 0.17282771374946707,
 ('thaw', 'melt'): 0.6608640736719353,
 ('see', 'meet'): 0.39826938719881827,
 ('claim', 'take'): 0.6465763408948699,
 ('sting', 'prick'): 0.9511679702174884,
 ('search', 'seek'): 0.13121050466442055,
 ('alter', 'change'): 0.41096219110187693,
 ('disturb', 'interrupt'): 0.8515749782698093,
 ('seem', 'appear'): 0.050970765621743706,
 ('have', 'hold'): 0.41973824972993606,
 ('travel', 'move'): 0.25494720510092705,
 ('travel', 'go'): -0.13169685882449686,
 ('start', 'begin'): 0.7213850412203451,
 ('prepare', 'cook'): 0.15593694756601564,
 ('let', 'allow'): 0.14751494765281137,
 ('look', 'appear'): 0.21810302476522653,
 ('float', 'drift'): 0.4464774246681577,
 ('suppose', 'guess'): 0.7175731631739495,
 ('call', 'yell'): -0.4170203345929416,
 ('associa

## Balanced accuracy

In [16]:
# The balanced_accuracy function needs as input a vector_dictionary object, a verb_similarity dictionary,
# two labels to be compared ad the type of product. The output is the balanced accuracy computed as
# described in the paper, using the full datset to define the divide and then to compute the accuracy
mf.balanced_accuracy(objdict,testsym,'ANTONYMS','SYNONYMS','std_dev')

0.5602808691043986

In [17]:
# It is also possible to compare labels grouped together, for example what is the accuracy when comparing synonyms
# with antonyms and none consideered as a single type
mf.balanced_accuracy(objdict,testsym,['ANTONYMS','NONE'],'SYNONYMS','std_dev')

0.5506936289337272

In [18]:
# Finally we can also compute the balanced accuracy by using a subset of the word pairs to define the divide
# and then testing on the remaining pairs. This is done with balanced_accuracy_subsets
# Inputs are a vector_dictionary object, a similarity_dictionary object, the labels to be compared, the product type,
# a seed for the random generator (for repeatable results) and the size of the "training samples" for each label
# Again it is possible to group labels together as shown for antonyms and none here
# The procedure is repeated on 20 different subsets (see paper), output is mean and standard deviation on the accuracy
mf.balanced_accuracy_subsets(objdict,testsym,['ANTONYMS','NONE'],'SYNONYMS','std_dev',7,72+1360,198)

array([0.54950225, 0.01570413])